In [1]:
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import generative_adversarial as gan
import dwdii_transforms
import numpy as np
import random

Using Theano backend.


In [4]:
import theano

#theano.config.floatX = 'float32'

print theano.config.device
print theano.config.floatX
print theano.config.mode
print theano.config.openmp
print theano.config.allow_gc
#print os.environ
#print(theano.config)

cpu
float32
FAST_RUN
True
False
{'DOCKERCLOUD_SERVICE_FQDN': 'one-b3b5deb8.630e6c21.svc.dockerapp.io', 'DOCKERCLOUD_NODE_HOSTNAME': '3d6e964a-0924-4643-b2f4-f66cf058718d', 'PATH': '/root/torch/install/bin:/root/caffe/build/tools:/root/caffe/python:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'HOME': '/root', 'LD_LIBRARY_PATH': '/root/torch/install/lib:', 'DOCKERCLOUD_NODE_API_URI': '/api/infra/v1/dwdii/node/3d6e964a-0924-4643-b2f4-f66cf058718d/', 'TERM': 'xterm-color', 'DOCKERCLOUD_NODE_API_URL': 'https://cloud.docker.com/api/infra/v1/dwdii/node/3d6e964a-0924-4643-b2f4-f66cf058718d/', 'DOCKERCLOUD_CONTAINER_HOSTNAME': 'one-b3b5deb8-1', 'DOCKERCLOUD_SERVICE_API_URL': 'https://cloud.docker.com/api/app/v1/dwdii/service/630e6c21-8364-4b48-b38a-1bd816a4a39e/', 'DOCKERCLOUD_SERVICE_API_URI': '/api/app/v1/dwdii/service/630e6c21-8364-4b48-b38a-1bd816a4a39e/', 'DOCKERCLOUD_STREAM_HOST': 'wss://ws.cloud.docker.com', 'LUA_PATH': '/root/.luarocks/share/lua/5.1/?.lua;/root/.luaroc

In [ ]:
theano.config.device = 'gpu'

In [ ]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"

### Load Data
In this section we load the image data and scale to 34x34.

In [ ]:
maxData = 2200
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = None, verboseFreq = 200, imgResize=(34,34), theseEmotions=["happiness"])
print X_data.shape
print Y_data.shape

In [ ]:
X_train, X_test, Y_train, Y_test = dwdii_transforms.trainingTestSplit(X_data, Y_data, 0.8, True)
shp = X_train.shape[1:]
print "Shape: " + str(shp)

### Create Generative and Discriminative Models

In [ ]:
genModel = gan.buildGenerativeModel(verbose=False)

In [ ]:
discrimModel = gan.buildDiscriminativeModel(shape=shp, verbose=False)

### Discriminator Training

In [ ]:
ntrain = 600
trainidx = random.sample(range(0,X_train.shape[0]), ntrain)
XT = X_train[trainidx,:,:,:]

In [ ]:
# Pre-train the discriminator network ...
noise_gen = np.random.uniform(0,1,size=[XT.shape[0],100])
generated_images = genModel.predict(noise_gen)
X = np.concatenate((XT, generated_images))
n = XT.shape[0]
y = np.zeros([2*n,2])
y[:n,1] = 1
y[n:,0] = 1

In [ ]:
gan.makeTrainable(discrimModel,True)
discrimModel.fit(X,y, nb_epoch=1, batch_size=128)

In [ ]:
y_hat = discrimModel.predict(X)

In [ ]:
# Measure accuracy of pre-trained discriminator network
y_hat_idx = np.argmax(y_hat,axis=1)
y_idx = np.argmax(y,axis=1)
diff = y_idx-y_hat_idx
n_tot = y.shape[0]
n_rig = (diff==0).sum()
acc = n_rig*100.0/n_tot
print "Accuracy: %0.02f pct (%d of %d) right"%(acc, n_rig, n_tot)

### Stacked GAN and Training

In [ ]:
loadWeights = True
if loadWeights:
    genModel.load_weights("dwdii-genModel-03-01.hdf5")
    discrimModel.load_weights("dwdii-discrimModel-03-01.hdf5")

# Build the stacked GAN model
GAN = gan.buildStackedGanModel(genModel, discrimModel)
if loadWeights:
    GAN.load_weights("dwdii-GAN-03-01.hdf5")

In [ ]:
gan.makeTrainable(discrimModel,False)

In [ ]:
gan.plotGenerative(0, genModel)

In [ ]:
# Train for N epochs at original learning rates
gan.train_for_n(X_train, GAN, genModel, discrimModel, nb_epoch=6000, plt_frq=50,BATCH_SIZE=32)

In [ ]:
ver = "03-02"
GAN.save_weights("dwdii-GAN-" + ver + ".hdf5", overwrite=True)
discrimModel.save_weights("dwdii-discrimModel-" + ver + ".hdf5", overwrite=True)
genModel.save_weights("dwdii-genModel-" + ver + ".hdf5", overwrite=True)